#### Imports


In [7]:
import pandas as pd
pd.set_option('display.max_colwidth', None)


In [8]:
from sentence_transformers import InputExample, losses, models, SentenceTransformer, util


In [9]:
from torch.utils.data import DataLoader
from torch import device, cuda, save, load

In [10]:
device = device('cuda' if cuda.is_available() else 'cpu')
device

device(type='cuda')

In [11]:
model = SentenceTransformer('all-mpnet-base-v2')
# telling our model to run on the gpu instead of cpu
model.to(device)

SentenceTransformer(
  (0): Transformer({'max_seq_length': 384, 'do_lower_case': False}) with Transformer model: MPNetModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False})
  (2): Normalize()
)

In [12]:
firefox_df= pd.read_csv('../fine-tuning-experimentation/data/firefox.csv', index_col='bug_id')

#### Encode the dataset

In [13]:
encoded_desc = []
report_id_list = []
path = 'firefox_encoded.csv'
num = 0
for index, row in firefox_df.iterrows(): # index refers to the index label of the current row, i.e. the bug_id
    try:
        encoded = model.encode(row[4])
    except:
        encoded = None
    #print(num)
    report_id = index
    encoded_desc.append(encoded)
    report_id_list.append(report_id)
    #num = num + 1

firefox_encoded_df = pd.DataFrame()
firefox_encoded_df["encoded_desc"] = encoded_desc
firefox_encoded_df["bug_id"] = report_id_list

    

C:\Users\rober\AppData\Local\Temp\ipykernel_35364\240678284.py:7: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  encoded = model.encode(row[4])
C:\Users\rober\AppData\Local\Temp\ipykernel_35364\240678284.py:7: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  encoded = model.encode(row[4])


In [ ]:
firefox_encoded_df.head(5)

#### Convert encoded dataset to CSV

In [ ]:
firefox_encoded_df.to_csv(path)


#### Convert encoded dataset to dict and save to pytorch file

In [ ]:
enc_dict = firefox_encoded_df.to_dict()


In [ ]:
report1 = enc_dict['encoded_desc'][0]
report2 = enc_dict['encoded_desc'][1]
#print(report1)
cossim = util.cos_sim(report1, report2)
print(cossim)

In [ ]:
path = 'enc_dict_test.pt'
save(enc_dict, path)

In [ ]:
loaded_enc_dict = load('enc_dict_test.pt')

In [ ]:
load_enc1 = loaded_enc_dict['encoded_desc'][0]
load_enc2 = loaded_enc_dict['encoded_desc'][1]
cossim = util.cos_sim(load_enc1, load_enc2)
print(cossim)